# Titers for a serum
Analyze titers for a serum, aggregating replicates which may be across multiple plates.

In [1]:
import altair as alt

import pandas as pd

_ = alt.data_transformers.disable_max_rows()

Get variables from `snakemake`:

In [2]:
######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/Users/jbloom/miniconda3/envs/seqneut-pipeline/lib/python3.11/site-packages', '/Users/jbloom/Library/Caches/snakemake/snakemake/source-cache/runtime-cache/tmpdr3i_bim/file/Users/jbloom/Library/CloudStorage/OneDrive-FredHutchinsonCancerCenter/seqneut-pipeline/notebooks', '/Users/jbloom/Library/CloudStorage/OneDrive-FredHutchinsonCancerCenter/seqneut-pipeline/notebooks']); import pickle; snakemake = pickle.loads(b"\x80\x04\x95i\x0f\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c$results/plates/plate11/curvefits.csv\x94\x8c#results/plates/plate2/curvefits.csv\x94e}\x94(\x8c\x06_names\x94}\x94\x8c\nplate_fits\x94K\x00K\x02\x86\x94s\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh\x13\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h\x19)}\x94\x8c\x05_name\x94h\x13sNt\x94bh\x14h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x14sNt\x94bh\x0fh\x06\x8c\tNamedlist\x94\x93\x94)\x81\x94(h\nh\x0be}\x94(h\r}\x94h\x11]\x94(h\x13h\x14eh\x13h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x13sNt\x94bh\x14h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x14sNt\x94bubub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94(\x8c,results/sera/M099d0/titers_per_replicate.csv\x94\x8c%results/sera/M099d0/titers_median.csv\x94e}\x94(h\r}\x94(\x8c\x0eper_rep_titers\x94K\x00N\x86\x94\x8c\rmedian_titers\x94K\x01N\x86\x94uh\x11]\x94(h\x13h\x14eh\x13h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x13sNt\x94bh\x14h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x14sNt\x94bh9h5h;h6ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94]\x94(\x8c\x14A/California/07/2009\x94\x8c\x12A/Michigan/45/2015\x94\x8c\x12A/Brisbane/02/2018\x94\x8c\x11A/Ghana/2080/2020\x94\x8c\x18A/Cote_DIvoire/1448/2021\x94\x8c\x10A/Ghana/138/2020\x94\x8c\x0fA/Togo/845/2020\x94\x8c\x10A/Togo/0274/2021\x94\x8c\x10A/Hawaii/70/2019\x94\x8c\x12A/Niger/10217/2021\x94\x8c\x19A/SouthAfrica/R16462/2021\x94\x8c\x10A/Togo/0304/2021\x94\x8c\x12A/Paris/31196/2021\x94\x8c\x14A/Belgium/H0017/2022\x94\x8c\x18A/England/220200318/2022\x94\x8c\x12A/Paris/30353/2021\x94\x8c\x10A/Nimes/871/2021\x94\x8c\x14A/Belgium/H0038/2022\x94\x8c\x14A/Wisconsin/588/2019\x94\x8c\x14A/Washington/23/2020\x94\x8c\x1aA/India/Pun-NIV312851/2021\x94\x8c\x1aA/India-PUN-NIV328484/2021\x94\x8c\x16A/Bangladesh/8036/2021\x94\x8c\x0eA/Perth/1/2022\x94\x8c\x19A/SouthAfrica/R14850/2021\x94\x8c\x13A/Norway/25089/2022\x94\x8c\x1cA/Bangladesh/3210810034/2021\x94\x8c\x13A/Chester/5355/2022\x94\x8c\x12A/Michigan/19/2021\x94\x8c\x0eA/Utah/27/2022\x94\x8c\x12A/Newcastle/2/2022\x94\x8c\x1fA/India-Pune-Nivcov2221170/2022\x94\x8c\x10A/Sydney/43/2022\x94\x8c\x12A/Brisbane/48/2022\x94\x8c\x16A/Bangladesh/8002/2021\x94\x8c\x16A/Bangladesh/2221/2021\x94ea}\x94(h\r}\x94\x8c\x17viral_strain_plot_order\x94K\x00N\x86\x94sh\x11]\x94(h\x13h\x14eh\x13h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x13sNt\x94bh\x14h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x14sNt\x94bhqhJub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x06M099d0\x94a}\x94(h\r}\x94\x8c\x05serum\x94K\x00N\x86\x94sh\x11]\x94(h\x13h\x14eh\x13h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x13sNt\x94bh\x14h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x14sNt\x94b\x8c\x05serum\x94h\x80ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c0/var/folders/jj/t2kjg1p146x_z68z7j80z8yw0000gq/T\x94e}\x94(h\r}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh\x11]\x94(h\x13h\x14eh\x13h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x13sNt\x94bh\x14h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x14sNt\x94bh\x97K\x01h\x99K\x01h\x9bh\x94ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c'results/sera/M099d0/titers_M099d0.ipynb\x94a}\x94(h\r}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh\x11]\x94(h\x13h\x14eh\x13h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x13sNt\x94bh\x14h\x17h\x19\x85\x94R\x94(h\x19)}\x94h\x1dh\x14sNt\x94bh\xadh\xaaub\x8c\x06config\x94}\x94(\x8c\x10seqneut-pipeline\x94\x8c\x03../\x94\x8c\x0fviral_libraries\x94}\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c-data/viral_libraries/pdmH1N1_lib2023_loes.csv\x94s\x8c\x17viral_strain_plot_order\x94\x8c data/viral_strain_plot_order.csv\x94\x8c\x12neut_standard_sets\x94}\x94\x8c\x08loes2023\x94\x8c3data/neut_standard_sets/loes2023_neut_standards.csv\x94s\x8c\x1eillumina_barcode_parser_params\x94}\x94(\x8c\x08upstream\x94\x8c\x1fCTCCCTACAATGTCGGATTTGTATTTAATAG\x94\x8c\ndownstream\x94\x8c\x00\x94\x8c\x04minq\x94K\x14\x8c\x11upstream_mismatch\x94K\x04\x8c\x0ebc_orientation\x94\x8c\x02R2\x94u\x8c$default_process_counts_qc_thresholds\x94}\x94(\x8c\x12avg_barcode_counts\x94K\xfa\x8c\x16min_neut_standard_frac\x94G?tz\xe1G\xae\x14{\x8c\x1fmax_neut_standard_frac_no_serum\x94G?\xb9\x99\x99\x99\x99\x99\x9a\x8c\x18barcode_frac_consistency\x94K\x04\x8c\x16min_viral_barcode_frac\x94G?@bM\xd2\xf1\xa9\xfc\x8c\x1emin_neut_standard_barcode_frac\x94G?tz\xe1G\xae\x14{\x8c\x17min_neut_standard_count\x94M^\x01\x8c min_no_serum_viral_barcode_count\x94K\x05\x8c!min_dilutions_per_serum_replicate\x94K\x04\x8c\x14max_frac_infectivity\x94K\x08u\x8c\x17default_curvefit_params\x94}\x94(\x8c\x18frac_infectivity_ceiling\x94K\x01\x8c\x06fixtop\x94\x89\x8c\tfixbottom\x94K\x00u\x8c\x06plates\x94}\x94(\x8c\x06plate2\x94}\x94(\x8c\x04date\x94\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe7\x08\x01\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate2_samples.csv\x94\x8c\x1cprocess_counts_qc_thresholds\x94}\x94(h\xd2K\xfah\xd3G?tz\xe1G\xae\x14{h\xd4G?\xb9\x99\x99\x99\x99\x99\x9ah\xd5K\x04h\xd6G?@bM\xd2\xf1\xa9\xfch\xd7G?tz\xe1G\xae\x14{h\xd8M^\x01h\xd9K\x05h\xdaK\x04h\xdbK\x08u\x8c\x10barcodes_to_drop\x94]\x94\x8c\x10GGTCCATCTCAGATCG\x94a\x8c\rwells_to_drop\x94]\x94(\x8c\x02C1\x94\x8c\x03D12\x94e\x8c\x0fcurvefit_params\x94}\x94(h\xdeK\x01h\xdf\x89h\xe0K\x00uu\x8c\x07plate11\x94}\x94(\x8c\x04date\x94h\xe8C\x04\x07\xe7\t\x1a\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1fdata/plates/plate11_samples.csv\x94\x8c\x1cprocess_counts_qc_thresholds\x94}\x94(h\xd2K\xfah\xd3G?tz\xe1G\xae\x14{h\xd4G?\xb9\x99\x99\x99\x99\x99\x9ah\xd5K\x04h\xd6G?@bM\xd2\xf1\xa9\xfch\xd7G?tz\xe1G\xae\x14{h\xd8M^\x01h\xd9K\x05h\xdaK\x04h\xdbK\x08u\x8c\x10barcodes_to_drop\x94]\x94\x8c\x10GGTCCATCTCAGATCG\x94a\x8c\rwells_to_drop\x94]\x94(\x8c\x02D6\x94\x8c\x03C12\x94e\x8c\x0fcurvefit_params\x94}\x94(h\xdeK\x01h\xdf\x89h\xe0K\x00uuuu\x8c\x04rule\x94\x8c\x0cserum_titers\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8ca/Users/jbloom/Library/CloudStorage/OneDrive-FredHutchinsonCancerCenter/seqneut-pipeline/notebooks\x94ub."); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir(r'/Users/jbloom/Library/CloudStorage/OneDrive-FredHutchinsonCancerCenter/seqneut-pipeline/test_example');
######## snakemake preamble end #########


In [3]:
plate_fits = snakemake.input.plate_fits
viral_strain_plot_order = snakemake.params.viral_strain_plot_order
per_rep_titers_csv = snakemake.output.per_rep_titers
median_titers_csv = snakemake.output.median_titers
serum = snakemake.wildcards.serum

Read all titers for this serum:

In [23]:
print(f"Reading titers for {serum=}")

assert len(plate_fits)

serum_fits = (
    pd.concat([pd.read_csv(f) for f in plate_fits])
    .query("serum == @serum")
    .drop(columns="serum")
)

assert len(serum_fits), f"no titers for {serum=}"
assert len(serum_fits) == len(serum_fits.groupby(["plate_replicate", "virus", "barcode"]))
print(f"Read a total of {len(serum_fits)} titers for {serum=}")

# get viruses in the order to plot them
viruses = sorted(serum_fits["virus"].unique())
if viral_strain_plot_order is not None:
    if not set(viruses).issubset(viral_strain_plot_order):
        raise ValueError(
            "`viral_strain_plot_order` lacks some viruses with titers:\n"
            + str(set(viruses) - set(viral_strain_plot_order))
        )
    viruses = [v for v in viral_strain_plot_order if v in viruses]
print(f"{serum=} has titers for a total of {len(viruses)} viruses")

Reading titers for serum='M099d0'
Read a total of 220 titers for serum='M099d0'
serum='M099d0' has titers for a total of 36 viruses


Get the NT50s per replicate.
Note that a lower bound in the IC50s represents and upper-bound on the NT50s:

In [21]:
per_rep_titers = (
    serum_fits
    .assign(
        nt50_bound=lambda x: x["ic50_bound"].map(
            {"interpolated": "interpolated", "upper": "lower", "lower": "upper"}
        ),
        serum=serum,
    )
    .rename(columns={"plate_replicate": "replicate"})
    [["serum", "replicate", "virus", "barcode", "nt50", "nt50_bound"]]
)

assert per_rep_titers.notnull().all().all()

per_rep_titers

,serum,replicate,virus,barcode,nt50,nt50_bound
0,M099d0,plate11,A/Bangladesh/2221/2021,AAGAGAAATATTCGCT,558.8,interpolated
1,M099d0,plate11,A/Bangladesh/2221/2021,GACAGCAATGCATACA,231.4,interpolated
2,M099d0,plate11,A/Bangladesh/2221/2021,GATAGAAATACCAGGA,538.8,interpolated
3,M099d0,plate11,A/Bangladesh/3210810034/2021,AACTATAGATCTAGAA,584.3,interpolated
4,M099d0,plate11,A/Bangladesh/3210810034/2021,ACAAAAGTACCTCTAC,185.3,interpolated
...,...,...,...,...,...,...
105,M099d0,plate2,A/Washington/23/2020,GAACTGGCGTCAATCA,213.0,interpolated
106,M099d0,plate2,A/Washington/23/2020,GGATAAGAAAACTACT,211.6,interpolated
107,M099d0,plate2,A/Wisconsin/588/2019,AGTCCTATCCTCAAAT,194.0,interpolated
108,M099d0,plate2,A/Wisconsin/588/2019,GAAGAAACTATAACCA,271.3,interpolated


Get the median titers. We defined the bounds for the median as the same as the bounds for the median measurement if there are an odd number of measurements, or if there are an even number of measurements interpolated only if all of the middle values are interpolated:

In [35]:
def get_median_nt50_bound(s):
    """Get the bound for the median NT50."""
    s = list(s)
    if len(s) % 2:
        return s[len(s) // 2]
    else:
        bounds = s[len(s) // 2 - 1: len(s) // 2 + 1]
        assert len(bounds) == 2
        if len(set(bounds)) == 1:
            return bounds[0]
        elif "interpolated" in bounds:
            return [b for b in bounds if b != "interpolated"][0]
        else:
            return "inconsistent"

median_titers = (
    per_rep_titers
    .sort_values("nt50")  # for getting median nt50 bound
    .groupby(["serum", "virus"], as_index=False)
    .aggregate(
        nt50=pd.NamedAgg("nt50", "median"),
        n_replicates=pd.NamedAgg("replicate", "count"),
        nt50_stdev=pd.NamedAgg("nt50", "std"),
        nt50_bound=pd.NamedAgg("nt50_bound", get_median_nt50_bound),
    )
)

median_titers

,serum,virus,nt50,n_replicates,nt50_stdev,nt50_bound
0,M099d0,A/Bangladesh/2221/2021,221.10,6,180.880142,interpolated
1,M099d0,A/Bangladesh/3210810034/2021,196.95,4,195.715208,interpolated
2,M099d0,A/Bangladesh/8002/2021,292.20,6,288.460801,interpolated
3,M099d0,A/Bangladesh/8036/2021,285.30,6,156.180161,interpolated
4,M099d0,A/Belgium/H0017/2022,566.15,6,94.957815,interpolated
5,M099d0,A/Belgium/H0038/2022,820.10,6,496.001480,interpolated
6,M099d0,A/Brisbane/02/2018,703.40,8,720.562542,interpolated
7,M099d0,A/Brisbane/48/2022,210.75,6,155.136852,interpolated
8,M099d0,A/California/07/2009,649.15,8,215.216216,interpolated
9,M099d0,A/Chester/5355/2022,386.80,6,207.076801,interpolated
